# What Makes a Great Board Game?

## Introduction
In this notebook, I will be exploring a dataset containing all board games and expansions showcased and rated in BoardGameGeek's website. I came across this dataset published by Gabriele Baldassarre on Kaggle, which you can find here: https://www.kaggle.com/gabrio/board-games-dataset

I will be using the CRISP-DM process to conduct my exploratory analysis. 

## Business Understanding
The goal of this analysis is to explore *what makes a great board game?* To do this, we will answer 3 business questions: 
1. What correlates to a board game's rating?
2. How does the quality of a board game change based on it's complexity?
3. Are complex board games considered popular? 

## Data Understanding

### Loading the Data
We will be using the pandas and matplotlib libraries to explore the data. We will also load the sqlite3 library to read the database and convert it into a pandas dataframe. 

In [ ]:
#Load the libraries

import numpy as np 
import pandas as pd 
import sqlite3 # to read the database
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [ ]:
#Load the Data
conn = sqlite3.connect(r'C:\Users\mitra\database.sqlite', uri=True)
df = pd.read_sql_query("SELECT * FROM BoardGames", conn)

#Verify that the result of SQL query is stored as a pandas DataFrame
df.head()

In [ ]:
#View the shape of the original dataset
print(df.shape)

## Data Preperation
The original dataset has over 90000 rows and 81 columns, that's definitely a lot! 

First, we are going to filter out board games and board game expansions that have 10 or less total ratings. This helps to clean the data as well as filter out homebrew content, variant gameplays and forgettable board games in general.  

We will then remove the null values in our most important stat - the board game rating i.e. 'stats.average'. After that, we will remove the columns that are not necessary for our exploratory analysis as well as null columns. 

Particularly, we will filter out boardgame entries only and then remove the 'game.type' column, as we only want to explore board games and not their expansions. We can assume that the expansion is dependent on its original, and hence is a biased column. 

In [ ]:
#Remove the null ratings in the dataset
cleaned_df = df[df['stats.average']> 0.0]
#Remove the entries with 5 or less total user ratings
cleaned_df = cleaned_df[cleaned_df['stats.usersrated']>10.0]
#Filter the dataset to only contain board games
cleaned_df = cleaned_df[cleaned_df['game.type']==boardgame]
#Remove all the unnecessary columns and null columns 
cleaned_df = cleaned_df.drop(['row_names','details.description','details.image','details.thumbnail',
                         'attributes.boardgamecompilation', 'attributes.boardgameexpansion', 'attributes.boardgamefamily',
                          'attributes.boardgameimplementation', 'attributes.boardgameintegration', 
                          'polls.language_dependence', 'polls.suggested_numplayers.1',
       'polls.suggested_numplayers.10', 'polls.suggested_numplayers.2',
       'polls.suggested_numplayers.3', 'polls.suggested_numplayers.4',
       'polls.suggested_numplayers.5', 'polls.suggested_numplayers.6',
       'polls.suggested_numplayers.7', 'polls.suggested_numplayers.8',
       'polls.suggested_numplayers.9', 'polls.suggested_numplayers.Over',
       'polls.suggested_playerage', 'attributes.t.links.concat.2....',
       'stats.family.amiga.bayesaverage', 'stats.family.amiga.pos',
       'stats.family.arcade.bayesaverage', 'stats.family.arcade.pos',
       'stats.family.atarist.bayesaverage', 'stats.family.atarist.pos',
       'stats.family.commodore64.bayesaverage', 'stats.family.commodore64.pos',
       'stats.subtype.rpgitem.bayesaverage', 'stats.subtype.rpgitem.pos',
       'stats.subtype.videogame.bayesaverage', 'stats.subtype.videogame.pos', 'stats.numcomments',
       'stats.numweights', 'stats.stddev',
       'stats.subtype.boardgame.bayesaverage', 'stats.subtype.boardgame.pos',
       'stats.trading', 'stats.wanting', 'stats.wishing', 'stats.family.abstracts.bayesaverage',
                         'stats.family.abstracts.pos', 'stats.family.cgs.bayesaverage', 'stats.family.cgs.pos',
                          'stats.family.childrensgames.bayesaverage' , 'stats.family.childrensgames.pos',
                          'stats.family.familygames.bayesaverage','stats.family.familygames.pos',
                          'stats.family.partygames.bayesaverage','stats.family.partygames.pos',
                          'stats.family.strategygames.bayesaverage' , 
                          'stats.family.strategygames.pos','stats.family.thematic.bayesaverage',
                          'stats.family.thematic.pos','stats.family.wargames.bayesaverage','stats.family.wargames.pos',
                         'attributes.boardgameartist', 'stats.median', 'attributes.total', 'stats.bayesaverage', 'game.type'], axis = 1)
cleaned_df.head()

In [ ]:
#View the shape of the cleaned dataset
print(cleaned_df.shape)

In [ ]:
#View the proportions of null values in the cleaned dataset 
cleaned_df.isnull().mean()

Looking at the proportions of null values in the cleaned dataset, we can see that the attribute columns have the most null values (albeit not many), while the detail columns have very few missing values. On the other hand, the stats and game name columns have no missing values, which are ultimately our most important columns for this dataset. 

Therefore, we will not be dropping any more rows or columns from this dataset, but we will be imputing values for the details columns using the mode. As the attribute columns are strings, we will not be imputing or removing their missing values.

We will also be renaming some columns for ease of analysis. 

In [ ]:
#Rename some columns for ease of analysis 
cleaned_df = cleaned_df.rename(columns = {"stats.average":"average rating", "stats.averageweight":"game complexity", 
                                          "stats.owned":"total owners", "stats.usersrated":"total ratings",
                                         "details.name":"game name"})
cleaned_df.head()

In [ ]:
#Imputing missing values in the detail columns with their respective mode in each column 
cleaned_df['details.maxplayers'].fillna(cleaned_df['details.maxplayers'].mode()[0], inplace=True)
cleaned_df['details.maxplaytime'].fillna(cleaned_df['details.maxplaytime'].mode()[0], inplace=True)
cleaned_df['details.minage'].fillna(cleaned_df['details.minage'].mode()[0], inplace=True)
cleaned_df['details.minplayers'].fillna(cleaned_df['details.minplayers'].mode()[0], inplace=True)
cleaned_df['details.minplaytime'].fillna(cleaned_df['details.minplaytime'].mode()[0], inplace=True)
cleaned_df['details.playingtime'].fillna(cleaned_df['details.playingtime'].mode()[0], inplace=True)
cleaned_df['details.yearpublished'].fillna(cleaned_df['details.yearpublished'].mode()[0], inplace=True)
cleaned_df.isnull().mean()

## Introduction to the Data: The Reign of Board Games 
Before we delve further into the data's features and particular business questions, we are going to explore how board games and its popularity progressed over the last 50 years. We will also explore the average rating distribution of board games in this dataset.

In [ ]:
#View the trend line graph of published board games over the last 50 years 
rslt_df = cleaned_df[cleaned_df['details.yearpublished']> 1959.0]
rslt_df = rslt_df[rslt_df['details.yearpublished']<2016.0]
timeline_df = rslt_df['details.yearpublished'].value_counts()
timeline_df.plot(kind = 'line', color = 'purple', xlabel="Year", ylabel="Total Board Games Published", 
                      title = "Published Board Games 1960-2016");

In [ ]:
#View the trend line graph of the average rating of published board games over the last 50 years 
rate_df = rslt_df.groupby('details.yearpublished').mean()['average rating']
rate_df.plot(kind = 'line', color = 'purple', xlabel="Year", ylabel="Average Rating of Board Games", 
                      title = "Average Rating of Board Games 1960-2016");

In [ ]:
#View the histogram of average board game ratings over the last 50 years
fig, ax = plt.subplots(figsize=(5,5))
rating_df = rslt_df['average rating']
rating_df.plot(kind = 'hist', alpha=0.5, bins=20, color = 'purple', title = 'Histogram of Board Game Ratings');

## 1. What Correlates to a Board Game's Rating? 
In this section, we will explore a correlation heatmap that looks into the features of this dataset and how they are correlated. We particularly want to look at the average rating feature, but also analyse other striking visuals in this plot. 

In this case, we filter the dataset to have more than a 100 ratings in total. This is to create a more unbiased sample, as some of these columns are estimated entries inputted by the BoardGameGeek community. 

In [ ]:
corr_df = cleaned_df[cleaned_df['total ratings']> 100.0]
corr = corr_df.corr()
fig, ax = plt.subplots(figsize=(10,10))
ax = sns.heatmap(
    corr, 
    vmin=0, vmax=1, center=0.5,
    cmap="YlGnBu",
    square=True, annot = True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

## 2. How Does a Game's Complexity Affect its Rating? 
Based on the findings from the previous section, we will now explore the average rating of games based on their respective complexity through a scatter plot.

We will filter out board games that do not have a complexity score tied to them, as this is most probably caused by an absense of information on the website itself. 

In [ ]:
#View the scatter plot of the average ratings of board games against their respective complexity score.
complex_df = cleaned_df[cleaned_df['game complexity']> 0.0]
complexity_df.plot(kind = 'scatter', x='game complexity', y='average rating', color = 'purple',
                title = "Scatter Plot of Board Games According to Complexity and Rating");


## 3. Is a Complex Game Considered Popular? 
We have seen a high correlation in the complexity of a board game towards the average rating, and we have seen a slight correlation in the total number of owners towards the average rating. 

In our final question, we will explore if a popular game is a highly rated game or not, and what the correlation is between complexity and popularity. Do they go hand in hand in making a great board game? Or does one feature triumph over the other in importance? 

In this section, we will be creating a new calculation to measure popularity, which is: (Average Rating)x(Total Ratings). This gives us the popularity score which we will explore with game complexity. 

In [ ]:
#Create a new column called 'popularity score 
cleaned_df['popularity score'] = cleaned_df['average rating'] * cleaned_df['total ratings']
cleaned_df.head()

In [ ]:
#Create a scatter plot on complexity versus popularity
pop_df = cleaned_df[cleaned_df['game complexity']> 0.0]
pop_df = pop_df[pop_df['total ratings']>100]
pop_df.plot(kind = 'scatter', x='game complexity', y='popularity score',
                title = "Scatter Plot of Game Complexity versus Game Popularity");